In [48]:
from PIL import Image, ImageDraw
import numpy as np
import math
from scipy import signal
from IPython.display import Image as Imag
import ncc
import os

In [7]:
os.getcwd()
os.chdir("./faces")

In [23]:
def MakePyramid(image,minsize):
    im = image
#   Get the x and y   
    x,y = im.size
    images = []
    while (x > minsize) and (y > minsize):
        new_img = im.resize((int(x*0.75), int(y*0.75)), Image.BICUBIC)
        images.append(new_img)
        x *= 0.75
        y *= 0.75
    return images
# makepyramid("./fans.jpg",10)

In [53]:
def ShowPyramid(pyramid):
     dx = 0
     dy = 0
     max_height = max([image.size[1] for image in pyramid])
     max_width = sum([image.size[0] for image in pyramid])
     new_image = Image.new("L", (max_width, max_height))    
     for im in pyramid:
        new_image.paste(im,(dx,dy))
        dx += im.size[0]
     new_image.show()
     
# ShowPyramid(makepyramid("./faces/fans.jpg",15))    
     
    

In [135]:
def FindTemplate(pyramid,template,threshold):
#   minimum possible template size.reduced template width as NCC is expensive
    template_w = 15
#   resize template to 
    scale = template.size[0]/template_w
    template = template.resize((template_w, template.size[1]//scale), Image.BICUBIC)
#   image must support color  
    x,y = template.size
    img = pyramid[0].convert('RGB')
#   loop over images in pyramid and get 
    for im in pyramid:
#       get all the pixels that are above threshold
        thresh = np.where(ncc.normxcorr2D(im, template) > threshold)
#       For each pixel at which the normalized correlation result 
#       is above the threshold, draw rectangle
        for i in range(len(thresh[1])):
                template_x = x/2
                template_y = y/2
                x1=thresh[1][i]-template_x
                y1=thresh[0][i]-template_y
                x2=thresh[1][i]+template_x
                y2=thresh[0][i]+template_y
                draw=ImageDraw.Draw(img)
                draw.rectangle((x1,y1,x2,y2),outline="red")
                display(Imag(filename='result.png'))
    img.save('result.png','PNG')
    img.show()            
    return img

In [147]:
allimages = ['family.jpg','fans.jpg','judybats.jpg','sports.jpg','students.jpg','tree.jpg',]
os.chdir("..")
os.chdir("./faces/")
template = Image.open("./template.jpg")
# The correlation threshold for whether the spot is a face or not.
threshold = 0.58

# For each image find the template in it.
for image in allimages:
    # Need to make a pyramid of images so we can match the template at multiple
    # zoom levels.
    image = str(os.getcwd()) +str(image)
    print image
    image = image.open(image)
    pyramid = MakePyramid(image, 10)
    # Find all template matches in the pyramid with threshold.
    result = FindTemplate(pyramid, template, threshold)
    # Output the found faces to terminal.
    print result

os.chdir("..")


/Users/Thenextbestthing/PycharmProjects/CPSC425/a2/facesfamily.jpg


AttributeError: 'str' object has no attribute 'open'